In [5]:
pip install scikit-learn pandas matplotlib seaborn numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [7]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data = Path('lending_data.csv')
lending_df = pd.read_csv(data)

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [8]:
# Get some information about our data
lending_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   borrower_income   77536 non-null  int64  
 3   debt_to_income    77536 non-null  float64
 4   num_of_accounts   77536 non-null  int64  
 5   derogatory_marks  77536 non-null  int64  
 6   total_debt        77536 non-null  int64  
 7   loan_status       77536 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 4.7 MB


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [9]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"].ravel()
# Separate the X variable, the features
x = lending_df.drop(columns="loan_status", axis=1)

/var/folders/wx/s3n4x4fj5bb91x6wzk5rn8v40000gn/T/ipykernel_4408/280103484.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y = lending_df["loan_status"].ravel()


In [10]:
# Review the y variable Series
print("y data(labels):")
y[0:10]

y data(labels):


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
# Review the X variable DataFrame
print("x data(features):")
x.head()

x data(features):


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


In [12]:
# Check the number of '0's(healthy loans) and '1's(high-risk loans) to check the balance of our target values
lending_df["loan_status"].value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [13]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test,  y_train, y_test = train_test_split(x, y, random_state = 1)
x_train.shape, x_test.shape

((58152, 7), (19384, 7))

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [14]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(solver='lbfgs', max_iter=200, random_state = 1)
# Fit the model using training data
model.fit(x_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [15]:
# Make a prediction using the testing data
y_predictions = model.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [16]:
# Generate a confusion matrix for the model
confusion_matrix = confusion_matrix(y_test, y_predictions)

# Create a DataFrame out of confusion matrix 
cm_df = pd.DataFrame(confusion_matrix, 
                     index = ["Actual 0", "Actual 1"],
                     columns = ["Prediction 0", "Prediction 1"]
                     )
cm_df

,Prediction 0,Prediction 1
Actual 0,18658,107
Actual 1,37,582


In [17]:
# Print the classification report for the model
print(classification_report(y_test, y_predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.94      0.89       619

    accuracy                           0.99     19384
   macro avg       0.92      0.97      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Our logistic regression model is doing an excellent job predicting the 0 class (healthy loans), with %100 precision, %99 recall. When it comes to risky loans (class 1), the model is still doing a good prediction, %85 precision and %91 recall. The lower percision and recall in predicting the risky loans (class 1) can be because of our imbalanced dataset as the support test data for class 1 is only 619 which is very low compared to 18765 for class 0. There is some room for improvement in predicting high risk loans.
The total accuracy of the model is high %99. 

---